In [39]:
import sys, os
sys.version, os.getcwd()

('3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]',
 '/home/jared/galvanize/Projects/prouster')

In [2]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split

/home/jared/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
!ls data/Oblique-Strategies-master/data/

obliquestrategies.csv


In [13]:
data = pd.read_csv('data/Oblique-Strategies-master/data/obliquestrategies.csv')

In [14]:
data.shape

(569, 3)

In [28]:
documents = data['card_text'].str.cat(sep = ' ')

In [29]:
with open('data/strings/OSs.txt', "w") as text_file:
    text_file.write(documents)

In [30]:
with open('data/strings/OSs.txt') as text_file:
    docs = text_file.read()

In [27]:
docs.count('\n')

0

In [6]:
vect = TfidfVectorizer(stop_words='english')
X = vect.fit_transform(documents)

In [7]:
X

<569x372 sparse matrix of type '<class 'numpy.float64'>'
	with 1557 stored elements in Compressed Sparse Row format>

In [4]:
data.head()

,card_id,edition,card_text
0,1,1,Abandon normal instruments
1,2,1,Accept advice
2,3,1,Accretion
3,4,1,A line has two sides
4,5,1,Allow an easement (an easement is the abandonm...


In [5]:
documents = data['card_text'].str.cat(sep = '. ')

In [17]:
rf = pd.read_csv('data/writingPrompts/RF.csv',header=None)

In [11]:
documents = rf[0].str.cat(sep = '. ')

In [18]:
len(rf)

1175

In [19]:
import string

In [20]:
chars = list(set(documents))
VOCAB_SIZE = len(chars)

In [21]:
chars

['H',
 'm',
 'W',
 'e',
 'T',
 'd',
 'i',
 'g',
 'A',
 '7',
 'z',
 '-',
 'v',
 'n',
 'b',
 'C',
 '(',
 'r',
 'Y',
 't',
 'w',
 'h',
 "'",
 '"',
 'K',
 'c',
 'I',
 ' ',
 'N',
 'j',
 '9',
 'u',
 'f',
 'B',
 'J',
 'k',
 'M',
 '.',
 'o',
 'y',
 '/',
 '&',
 '8',
 'a',
 ',',
 'p',
 'O',
 'D',
 '0',
 ')',
 'P',
 's',
 'x',
 'l',
 '1',
 'E',
 'U',
 'S']

In [22]:
VOCAB_SIZE

58

In [114]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}
char_to_ix

{'7': 0,
 'S': 1,
 'c': 2,
 '-': 3,
 'r': 4,
 'N': 5,
 'V': 6,
 '3': 7,
 '9': 8,
 'J': 9,
 'v': 10,
 'B': 11,
 '0': 12,
 ',': 13,
 '"': 14,
 '&': 15,
 'd': 16,
 's': 17,
 'O': 18,
 'l': 19,
 '4': 20,
 'W': 21,
 'o': 22,
 'F': 23,
 '’': 24,
 'x': 25,
 'U': 26,
 'Q': 27,
 'K': 28,
 'C': 29,
 'w': 30,
 'A': 31,
 "'": 32,
 'P': 33,
 '8': 34,
 ' ': 35,
 '1': 36,
 '5': 37,
 'H': 38,
 'i': 39,
 '6': 40,
 'u': 41,
 't': 42,
 'j': 43,
 'n': 44,
 'g': 45,
 'E': 46,
 '(': 47,
 'q': 48,
 'I': 49,
 ';': 50,
 '?': 51,
 'a': 52,
 'T': 53,
 'k': 54,
 ':': 55,
 '2': 56,
 'L': 57,
 'f': 58,
 'M': 59,
 ')': 60,
 '“': 61,
 '!': 62,
 'm': 63,
 'é': 64,
 'b': 65,
 '.': 66,
 'R': 67,
 '”': 68,
 'z': 69,
 '/': 70,
 'D': 71,
 'p': 72,
 '$': 73,
 'Y': 74,
 'e': 75,
 '%': 76,
 'G': 77,
 'y': 78,
 'h': 79}

In [115]:
len(data)//6

94

In [116]:
D_LENGTH = int(len(documents))

In [117]:
D_LENGTH

121058

In [118]:
SEQ_LENGTH = 200

In [119]:
D_LENGTH//SEQ_LENGTH

605

In [120]:
 
X = np.zeros((len(documents)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(documents)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(documents)//SEQ_LENGTH):
    X_sequence = documents[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = documents[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        print(j)
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82

81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
1

...

148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
3

147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
1

44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
13

In [121]:
X_sequence, X_sequence_ix, 

(' the car ride back from the hospital and neither of them quite knows how to talk about what just happened .. Write the most horrifying , or creepy , story you can think of that is completely based in ',
 [35,
  42,
  79,
  75,
  35,
  2,
  52,
  4,
  35,
  4,
  39,
  16,
  75,
  35,
  65,
  52,
  2,
  54,
  35,
  58,
  4,
  22,
  63,
  35,
  42,
  79,
  75,
  35,
  79,
  22,
  17,
  72,
  39,
  42,
  52,
  19,
  35,
  52,
  44,
  16,
  35,
  44,
  75,
  39,
  42,
  79,
  75,
  4,
  35,
  22,
  58,
  35,
  42,
  79,
  75,
  63,
  35,
  48,
  41,
  39,
  42,
  75,
  35,
  54,
  44,
  22,
  30,
  17,
  35,
  79,
  22,
  30,
  35,
  42,
  22,
  35,
  42,
  52,
  19,
  54,
  35,
  52,
  65,
  22,
  41,
  42,
  35,
  30,
  79,
  52,
  42,
  35,
  43,
  41,
  17,
  42,
  35,
  79,
  52,
  72,
  72,
  75,
  44,
  75,
  16,
  35,
  66,
  66,
  35,
  21,
  4,
  39,
  42,
  75,
  35,
  42,
  79,
  75,
  35,
  63,
  22,
  17,
  42,
  35,
  79,
  22,
  4,
  4,
  39,
  58,
  78,
  39,
  44,
  45,


In [122]:
y_sequence, y_sequence_ix

('the car ride back from the hospital and neither of them quite knows how to talk about what just happened .. Write the most horrifying , or creepy , story you can think of that is completely based in r',
 [42,
  79,
  75,
  35,
  2,
  52,
  4,
  35,
  4,
  39,
  16,
  75,
  35,
  65,
  52,
  2,
  54,
  35,
  58,
  4,
  22,
  63,
  35,
  42,
  79,
  75,
  35,
  79,
  22,
  17,
  72,
  39,
  42,
  52,
  19,
  35,
  52,
  44,
  16,
  35,
  44,
  75,
  39,
  42,
  79,
  75,
  4,
  35,
  22,
  58,
  35,
  42,
  79,
  75,
  63,
  35,
  48,
  41,
  39,
  42,
  75,
  35,
  54,
  44,
  22,
  30,
  17,
  35,
  79,
  22,
  30,
  35,
  42,
  22,
  35,
  42,
  52,
  19,
  54,
  35,
  52,
  65,
  22,
  41,
  42,
  35,
  30,
  79,
  52,
  42,
  35,
  43,
  41,
  17,
  42,
  35,
  79,
  52,
  72,
  72,
  75,
  44,
  75,
  16,
  35,
  66,
  66,
  35,
  21,
  4,
  39,
  42,
  75,
  35,
  42,
  79,
  75,
  35,
  63,
  22,
  17,
  42,
  35,
  79,
  22,
  4,
  4,
  39,
  58,
  78,
  39,
  44,
  45,
  35,


In [123]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, TimeDistributed, Activation, Dense, Lambda

In [124]:
HIDDEN_DIM = 300
LAYER_NUM = 4
BATCH_SIZE = 1000
temp = 1

In [125]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Lambda(lambda x: x / temp))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [126]:
 
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
 
nb_epoch = 0

print('\n\n')
model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=10)
nb_epoch += 1
generate_text(model, SEQ_LENGTH)
if nb_epoch % 2 == 0:
    model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))

/home/jared/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/20


In [ ]:
sample = generate_text(model, 10)

In [ ]:
documents.to_hdf('documents.h5', 'docs')

In [ ]:
docs = pd.read_hdf('documents.h5','docs')

In [79]:
all(docs == documents)

True

In [72]:
model.save('tg.h5')

In [ ]:
import numpy as np
from keras.models import Sequential, Model, Input, load_model
from keras.layers import LSTM, TimeDistributed, Activation, Dense

class TextGenerator:

    def __init__(self, model_file, documents_file, key='docs'):
        self.documents = pd.read_hdf(documents_file, key)
        self.model = load_model(model_file)
        self.VOCAB_SIZE = len(list(set(self.documents)))
        self.chars = list(set(documents))
        self.ix_to_char = {ix:char for ix, char in enumerate(chars)}

        
    def generate_text(self, length=15):
        ix = [np.random.randint(self.VOCAB_SIZE)]
        y_char = [self.ix_to_char[ix[-1]]]
        X = np.zeros((1, length, self.VOCAB_SIZE))
        for i in range(length):
            X[0, i, :][ix[-1]] = 1
            #print(self.ix_to_char[ix[-1]], end="")
            ix = np.argmax(self.model.predict(X[:, :i+1, :])[0], 1)
            y_char.append(self.ix_to_char[ix[-1]])
        return ('. ').join(y_char)+'.'
        

In [82]:
tg = TextGenerator('tg.h5','documents.h5')
tg.generate_text()

'Overtly resist change. Give the game away. Give the game away. Give the game away. Give the game away. Give the game away. Give the game away. Give the game away. Look at the order in which you do things. Listen to the quiet voice. Make a sudden, destructive unpredictable action; incorporate. Make a sudden, destructive unpredictable action; incorporate. Make a sudden, destructive unpredictable action; incorporate. Make an exhaustive list of everything you might do and do the last thing on the list. Make an exhaustive list of everything you might do and do the last thing on the list. Make an exhaustive list of everything you might do and do the last thing on the list.'

In [83]:
tg.generate_text()

"When is it for?. Ask people to work against their better judgement. Ask people to work against their better judgement. Ask people to work against their better judgement. Ask people to work against their better judgement. Ask people to work against their better judgement. Ask people to work against their better judgement. Ask people to work against their better judgement. Breathe more deeply. Breathe more deeply. Accept advice. Accept advice. Accept advice. Accept advice. Accept advice. Use 'unqualified' people."

In [70]:
type(tg)

__main__.TextGenerator

In [2]:
Response = "Whatever happens, be careful that it will work. We will never leave you. I have bacon ready for your ass."

In [3]:
import collections, itertools
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews, stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import *
from nltk.probability import FreqDist, ConditionalFreqDist
 
def evaluate_classifier(featx):
    negids = movie_reviews.fileids('neg')
    posids = movie_reviews.fileids('pos')
 
    negfeats = [(featx(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
    posfeats = [(featx(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
    negcutoff = len(negfeats)*3//4
    poscutoff = len(posfeats)*3//4
 
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
 
    classifier = NaiveBayesClassifier.train(trainfeats)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(testfeats):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
 
    print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
    classifier.show_most_informative_features()
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
print( 'evaluating single word features')
evaluate_classifier(word_feats)

evaluating single word features
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [4]:
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()
 
for word in movie_reviews.words(categories=['pos']):
    word_fd[word.lower()] +=1
    label_word_fd['pos'][word.lower()] +=1
 
for word in movie_reviews.words(categories=['neg']):
    word_fd[word.lower()] +=1
    label_word_fd['neg'][word.lower()] +=1
 
# n_ii = label_word_fd[label][word]
# n_ix = word_fd[word]
# n_xi = label_word_fd[label].N()
# n_xx = label_word_fd.N()
 
pos_word_count = label_word_fd['pos'].N()
neg_word_count = label_word_fd['neg'].N()
total_word_count = pos_word_count + neg_word_count

In [5]:
word_scores = {}
 
for word, freq in word_fd.items():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
        (freq, pos_word_count), total_word_count)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
        (freq, neg_word_count), total_word_count)
    word_scores[word] = pos_score + neg_score
 
best = sorted(word_scores.items())[:10000]
bestwords = set([w for w, s in best])
 
def best_word_feats(words):
    return dict([(word, True) for word in words if word in bestwords])
 
print('evaluating best word features')
evaluate_classifier(best_word_feats)
 
def best_bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    d = dict([(bigram, True) for bigram in bigrams])
    d.update(best_word_feats(words))
    return d

evaluating best word features
accuracy: 0.676
Most Informative Features
                  avoids = True              pos : neg    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
               affecting = True              pos : neg    =      9.7 : 1.0
                  annual = True              pos : neg    =      9.0 : 1.0
                bothered = True              neg : pos    =      9.0 : 1.0
               animators = True              pos : neg    =      9.0 : 1.0
                  azaria = True              neg : pos    =      8.3 : 1.0
                    anna = True              pos : neg    =      7.7 : 1.0
                 detract = True              pos : neg    =      7.7 : 1.0
                depicted = True              pos : neg    =      7.4 : 1.0


In [6]:
print('evaluating best words + bigram chi_sq word features')
evaluate_classifier(best_bigram_word_feats)

evaluating best words + bigram chi_sq word features
accuracy: 0.832
Most Informative Features
          ('give', 'us') = True              neg : pos    =     12.3 : 1.0
       ('matt', 'damon') = True              pos : neg    =     12.3 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
    ('absolutely', 'no') = True              neg : pos    =     10.6 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
       ('fairy', 'tale') = True              pos : neg    =     10.3 : 1.0
    ('quite', 'frankly') = True              neg : pos    =      9.7 : 1.0
               affecting = True              pos : neg    =      9.7 : 1.0
          ('does', 'so') = True              pos : neg    =      9.7 : 1.0
     ('saving', 'grace') = True              neg : pos    =      9.7 : 1.0


In [7]:
word_fd.items()

dict_items([('films', 1536), ('adapted', 46), ('from', 4999), ('comic', 389), ('books', 78), ('have', 4901), ('had', 1546), ('plenty', 134), ('of', 34123), ('success', 216), (',', 77717), ('whether', 217), ('they', 4825), ("'", 30585), ('re', 1132), ('about', 3523), ('superheroes', 12), ('(', 11664), ('batman', 203), ('superman', 26), ('spawn', 82), (')', 11781), ('or', 3148), ('geared', 14), ('toward', 100), ('kids', 328), ('casper', 23), ('the', 76529), ('arthouse', 3), ('crowd', 80), ('ghost', 77), ('world', 1037), ('but', 8634), ('there', 3770), ('s', 18513), ('never', 1374), ('really', 1558), ('been', 2050), ('a', 38106), ('book', 347), ('like', 3690), ('hell', 240), ('before', 992), ('.', 65876), ('for', 9961), ('starters', 8), ('it', 16107), ('was', 4940), ('created', 167), ('by', 6261), ('alan', 101), ('moore', 84), ('and', 35576), ('eddie', 149), ('campbell', 94), ('who', 5692), ('brought', 157), ('medium', 31), ('to', 31937), ('whole', 495), ('new', 1292), ('level', 270), ('i

In [ ]:
from py_thesaurus import Thesaurus

In [26]:
input_word = "dream"

new_instance = Thesaurus(input_word)

TypeError: __init__() missing 1 required positional argument: 'word'

In [29]:
# Get the synonyms according to part of speech
# Default part of speech is noun

print(new_instance.get_synonym())

print(new_instance.get_synonym(pos='verb'))

print(new_instance.get_synonym(pos='adj'))


# Get the antonyms

print(new_instance.get_antonym())

[]
[]
[]
[]


In [ ]:
from nltk import wordnet as wn

In [33]:
from keras im

In [39]:
!ls

appChar.py		   model.h5		      temp2keras.h5
appWord.py		   ObliqueStrategies-2.ipynb  templates
clean_reddit_data.py	   pickle_tokens.py	      textgeneratorN.py
clean_reddit_data_test.py  __pycache__		      textgenerator.py
data			   README.md		      tg.h5
data.tar.gz		   results		      tokenizer.pkl
documents.h5		   rNNcomparison.ipynb	      TorchDemo.ipynb
download_model.py	   sequence_text.py
get_pickle.py		   static


In [40]:
from textgeneratorN import WordGenerator

In [76]:
tg = WordGenerator(
    model_file='results/model.h5', 
    documents_file='results/sequences.txt',
    token_file='results/tokenizer.pkl',
    key='docs')

In [77]:
s = tg.generate_seq()
s

'the queen successfully takes back of the ghetto you are nt love to make it all children ve just gone kicked a winning political candidate is giving a speech when suddenly they stop my story like he never came home it was wanted cold but now they might be no'

In [78]:
if s[0:2] == 's ':
    s = s[2:] + '...'

In [80]:
s

'the queen successfully takes back of the ghetto you are nt love to make it all children ve just gone kicked a winning political candidate is giving a speech when suddenly they stop my story like he never came home it was wanted cold but now they might be no'